In [1]:
print("Hello")

Hello


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from huggingface_hub import notebook_login
import torch
import transformers

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
from langchain import HuggingFacePipeline

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

import os
import sys


/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

document=[]
for file in os.listdir("./patriz_test_2/"):
  if file.endswith(".pdf"):
    pdf_path="./patriz_test_2/"+file
    loader=PyPDFLoader(pdf_path)
    document.extend(loader.load())
  elif file.endswith('.docx') or file.endswith('.doc'):
    doc_path="./patriz_test_2/"+file
    loader=Docx2txtLoader(doc_path)
    document.extend(loader.load())
  elif file.endswith('.txt'):
    text_path="./patriz_test_2/"+file
    loader=TextLoader(text_path)
    document.extend(loader.load())

In [4]:
document

[Document(page_content='US006938300B2\nUSOO69383OOB2\n(12) United States Patent\nLan\nLan\n(54) WHEEL ASSEMBLY FOR A STROLLER\n(54) WHEEL ASSEMBLY FOR ASTROLLER\n(76) Inventor: Red Lan, 15F, No. 108, Sec. 1 Hsin Tai\n5th Rd., Hsichih, Taipei Hsien (TW)\n5th Rd., Hsichih, Taipei Hsien (TW)\n(*) Notice:\n(*) Notice:\nSubject to any disclaimer, the term of this\npatent is extended or adjusted under 35\nUS.C. 154(b) by 0 days.\nSubject to any disclaimer, the term of this\npatent is extended or adjusted under 35\nU.S.C. 154(b) by 0 days.\n(21) Appl. No.: 10/438,910\n(22) Filed:\n(21) Appl. No.: 10/438,910\n(22) Filed:\nMay 16, 2003\nMay 16, 2003\n(65)\n(65)\nPrior Publication Data\nPrior Publication Data\nUS 2004/0226133 A1 Nov. 18, 2004\nUS 2004/0226133 Al Nov. 18, 2004\n(51) Int. Cl. ............................ B60B33/00; B62B 7/04\n(52) U.S. Cl. ....................... 16/18 R; 16/35 R; 16/18 A;\n16/44; 16/31 R; 280/47.25; 280/642; 280/647;\n280/657; 280/47.38\n(58) Field of Search ....

In [5]:
len(document)

1

In [6]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=512, chunk_overlap=50)

In [7]:
document_chunks=document_splitter.split_documents(document)
len(document_chunks)

46

In [66]:
document_chunks[3]

Document(page_content='5,221,100 A\n5,232,071 A *\n6/1993 McNutt ....................... 280/78\nMcNutt oe280/78\n8/1993\n5,581,843 A\n12/1996\n5,581,843 A * 12/1996\n5,819,514 A 5,819,514 A * 10/1998\n10/1998\nKawanabe ...\nw. 188/1.12 ... 188/1.12\nPurnell ....... Purnell ....................... 16/35 R\nFirdaus .......................\nee 16/35 R\nFirdaus ......0... 56/16.7eee 56/16.7\n6/1993\n(51) Int. C1? eeeB60B 33/00; B62B 7/04\n(52) US. Ch oe16/18 R; 16/35 R; 16/18 A;\n16/44; 16/31 R; 280/47.25; 280/642; 280/647;\n280/657; 280/47.38', metadata={'source': './patriz_test_2/US6938300_patriz.txt'})

In [9]:
#Embedding model load

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    # ,
    # query_instruction="为这个句子生成表示以用于检索相关文章："
)


emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())

[codecarbon WARNING @ 16:48:12] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 16:48:12] [setup] RAM Tracking...
[codecarbon INFO @ 16:48:12] [setup] GPU Tracking...
[codecarbon INFO @ 16:48:12] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 16:48:12] [setup] CPU Tracking...
[codecarbon WARNING @ 16:48:12] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716824892.7502155


[codecarbon WARNING @ 16:48:14] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 16:48:14] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 16:48:14] >>> Tracker's metadata:
[codecarbon INFO @ 16:48:14]   Platform system: Linux-5.4.0-182-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 16:48:14]   Python version: 3.12.1
[codecarbon INFO @ 16:48:14]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 16:48:14]   Available RAM : 503.339 GB
[codecarbon INFO @ 16:48:14]   CPU count: 112
[codecarbon INFO @ 16:48:14]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 16:48:14]   GPU count: 2
[codecarbon INFO @ 16:48:14]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 16:48:19] Energy consumed for RAM : 0.000122 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 16:48:19] Energy consumed for all GPUs : 0.000102 kWh. Total GPU Power : 157.719756535

Emissions for this code execution is:  6.585105440103708e-05
End of code time:  1716824899.606445


In [10]:
print(model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-large-en-v1.5' cache_folder=None model_kwargs={} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction=''


In [11]:
#Chroma DB saving files and embeddings

from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

chroma_db = Chroma.from_documents(
    document_chunks, model, persist_directory="./db_patriz"
)

chroma_db.persist()

emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())


[codecarbon WARNING @ 16:48:30] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 16:48:30] [setup] RAM Tracking...
[codecarbon INFO @ 16:48:30] [setup] GPU Tracking...
[codecarbon INFO @ 16:48:30] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 16:48:30] [setup] CPU Tracking...
[codecarbon WARNING @ 16:48:30] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1716824910.2278676


[codecarbon WARNING @ 16:48:31] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 16:48:31] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 16:48:31] >>> Tracker's metadata:
[codecarbon INFO @ 16:48:31]   Platform system: Linux-5.4.0-182-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 16:48:31]   Python version: 3.12.1
[codecarbon INFO @ 16:48:31]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 16:48:31]   Available RAM : 503.339 GB
[codecarbon INFO @ 16:48:31]   CPU count: 112
[codecarbon INFO @ 16:48:31]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 16:48:31]   GPU count: 2
[codecarbon INFO @ 16:48:31]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 16:48:36] Energy consumed for RAM : 0.000089 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 16:48:36] Energy consumed for all GPUs : 0.000103 kWh. Total GPU Power : 217.674559177

Emissions for this code execution is:  5.5518979863521e-05
End of code time:  1716824916.4184031


In [12]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import BitsAndBytesConfig


In [24]:
model_id = "/home/c7361293/triz_llm/llama-2-7b-chat/model"
 
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
                 model_id,
                 device_map="cpu",
                 use_auth_token=True,
                #  load_in_8bit=True
                 )



/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.11it/s]


In [67]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.bfloat16,
              device_map='cpu',
              max_new_tokens=1024,
              min_new_tokens=-1,
              top_k=30
              )

In [68]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [69]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7fd77412d310>, model_kwargs={'temperature': 0})

In [70]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [71]:
#Create our Q/A Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=chroma_db.as_retriever(search_kwargs={'k':6}),
                                             verbose=False, memory=memory)

In [72]:
import torch
torch.cuda.empty_cache()

result=pdf_qa({"question":"""
    Quote the lines from the documents and present them directly.
    What are the line that contains improvement and obstacles of mentioned in the documents.
"""})
torch.cuda.empty_cache()
torch.cuda.mem_get_info()

/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:509: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `30` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


(2274754560, 47736619008)

In [73]:
result['answer']

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nret\n=<\n=\n—_—_—_\ns\nml\n14\n142\n}\nk\nU.S. Patent\nSep. 6, 2005\nSheet 3 of 9\nUS 6,938,300 B2\nFIG,3\nU.S. Patent\nSep. 6, 2005\nSheet 4 of 9\nUS 6,938,300 B2\nU.S. Patent\nSep. 6, 2005\nSheet 5 of 9\nUS 6,938,300 B2\n316°\n312\n613\n612\n65\nFIG.9\nwre ™ 4\n4\nU.S. Patent\nSep. 6, 2005\nSheet 6 of 9\nae eeeR\nUS 6,938,300 B2\nU.S. Patent\nSep. 6, 2005\nSheet 7 of 9\nUS 6,938,300 B2\nFIG. 7\nU.S. Patent\nSep. 6, 2005\nSheet 8 of 9\nUS 6,938,300 B2\nFIG. 8\nwre ™ 4\nU.S. Patent\nSep. 6, 2005\nSheet 9 of 9\nUS 6,938,300 B2\n643\nFIG.9\nUS 6,938,300 B2\n1\n\n25\ndeparting the Scope and Spirit of this invention. It is therefore\nintended that this invention be limited only as indicated by\nthe appended claims.\nI claim:\n30\n35\n40\n1. A Stroller having a stroller frame, a pair of front wheel\nassemblies disposed on a fr

In [74]:
text_input = result["answer"]
summary_index = text_input.find("Helpful Answer:\n")
summary = text_input
if summary_index != -1:
    # Extract the text from "Begin Summary:" to the end of the string
        summary = text_input[summary_index:]

print(summary.strip())

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

ret
=<
=
—_—_—_
s
ml
14
142
}
k
U.S. Patent
Sep. 6, 2005
Sheet 3 of 9
US 6,938,300 B2
FIG,3
U.S. Patent
Sep. 6, 2005
Sheet 4 of 9
US 6,938,300 B2
U.S. Patent
Sep. 6, 2005
Sheet 5 of 9
US 6,938,300 B2
316°
312
613
612
65
FIG.9
wre ™ 4
4
U.S. Patent
Sep. 6, 2005
Sheet 6 of 9
ae eeeR
US 6,938,300 B2
U.S. Patent
Sep. 6, 2005
Sheet 7 of 9
US 6,938,300 B2
FIG. 7
U.S. Patent
Sep. 6, 2005
Sheet 8 of 9
US 6,938,300 B2
FIG. 8
wre ™ 4
U.S. Patent
Sep. 6, 2005
Sheet 9 of 9
US 6,938,300 B2
643
FIG.9
US 6,938,300 B2
1

25
departing the Scope and Spirit of this invention. It is therefore
intended that this invention be limited only as indicated by
the appended claims.
I claim:
30
35
40
1. A Stroller having a stroller frame, a pair of front wheel
assemblies disposed on a front end of Said Stroller, and a pair
of rear wheel assemblies dispos

In [4]:
import torch

# del model

torch.cuda.empty_cache()
torch.cuda.mem_get_info()

NameError: name 'model' is not defined